# DATA WRANGLING - DEMAND CLUSTERING

In [1]:
import pandas as pd
import numpy as np

ruta_archivo = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras Anastasia\Carga Historia de Venta\2024-10 Ciclo Nov-24\AFM\Correccion Dispo\Anastasia 2024.10.csv"
df = pd.read_csv(ruta_archivo, delimiter=',', decimal=',', encoding='utf-8').rename(columns={'Canal': 'Canal 3', 'Cantidad': 'Venta', 'idSKU': 'Ultimo Eslabón'})
df['ID'] = df['Ultimo Eslabón'].astype(str) + df['Canal 3'].astype(str)
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d')
df = df.sort_values(by=['Ultimo Eslabón', 'Fecha']).reset_index(drop=True)
pivot_table = df.pivot_table(index=['Ultimo Eslabón', 'Canal 3'], columns='Fecha', values='Venta', fill_value=0).reset_index()
pivot_table = pivot_table[pivot_table.iloc[:, 2:].sum(axis=1) >= 2]
ultima_fecha = pivot_table.columns[-1]
primer_fecha = ultima_fecha - pd.DateOffset(months=23)
ventas_ultimos_24_meses = pivot_table.loc[:, primer_fecha:ultima_fecha]
intervalos = ventas_ultimos_24_meses.apply(lambda x: (x != 0).astype(int).diff().fillna(1).abs().sum(), axis=1)
ventas_activas = (ventas_ultimos_24_meses != 0).sum(axis=1)
pivot_table['ADI'] = intervalos / ventas_activas
ventas_no_cero = ventas_ultimos_24_meses.replace(0, np.nan)
pivot_table['CV²'] = (ventas_no_cero.std(axis=1) / ventas_no_cero.mean(axis=1)) ** 2
def clasificar_demanda(row):
    if row['ADI'] < 1.32 and row['CV²'] < 0.49: return 'Smooth'
    elif row['ADI'] >= 1.32 and row['CV²'] < 0.49: return 'Intermittent'
    elif row['ADI'] < 1.32 and row['CV²'] >= 0.49: return 'Erratic'
    else: return 'Lumpy'
pivot_table['Demand Type'] = pivot_table.apply(clasificar_demanda, axis=1)
melted_data = pivot_table.melt(id_vars=['Ultimo Eslabón', 'Canal 3', 'ADI', 'CV²', 'Demand Type'], var_name='Fecha', value_name='Venta')
melted_data['Fecha'] = pd.to_datetime(melted_data['Fecha'], format='%Y-%m-%d')
data = melted_data
data.shape

(4203414, 7)

# OPCIONAL GUARDAR COMO EXCEL EL CLUSTERING

In [4]:
# data['ID'] = data['Ultimo Eslabón'].astype(str) + data['Canal 3']
# data = data[['Ultimo Eslabón', 'ID', 'Demand Type']].drop_duplicates()
# ruta_guardado = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras Anastasia\Carga Historia de Venta\2024-10 Ciclo Nov-24\AFM\Correccion Dispo\Cluster 2024.10.xlsx"
# data.to_excel(ruta_guardado, index=False)
# data = melted_data

In [2]:
# Contar el número de combinaciones únicas de 'Ultimo Eslabón' y 'Canal 3' para cada 'Demand Type'
unique_counts = melted_data.groupby('Demand Type').apply(lambda x: x[['Ultimo Eslabón', 'Canal 3']].drop_duplicates().shape[0])

# Contar el número de filas para cada 'Demand Type'
filas_count = melted_data['Demand Type'].value_counts()

# Imprimir los resultados
print("Unique counts of ID & Canal 3 per Demand Type:")
print(unique_counts)

print("\nRow counts per Demand Type:")
print(filas_count)


Unique counts of ID & Canal 3 per Demand Type:
Demand Type
Erratic          4379
Intermittent     4328
Lumpy           32655
Smooth          10532
dtype: int64

Row counts per Demand Type:
Demand Type
Lumpy           2645055
Smooth           853092
Erratic          354699
Intermittent     350568
Name: count, dtype: int64


C:\Users\etorres.DERCOPARTS\AppData\Local\Temp\ipykernel_24512\1170459666.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_counts = melted_data.groupby('Demand Type').apply(lambda x: x[['Ultimo Eslabón', 'Canal 3']].drop_duplicates().shape[0])


# ARIMA SMOOTH #PRESENTE VIRTUAL

In [4]:
data = melted_data[melted_data['Demand Type'] == 'Erratic']
data = data[['Ultimo Eslabón', 'Canal 3', 'Fecha', 'Venta']]
data.head(2)

,Ultimo Eslabón,Canal 3,Fecha,Venta
5,100480,CL RETAIL,2018-01-01,159.0
9,100483,CL RETAIL,2018-01-01,0.0


In [4]:
data.head(1)

,Ultimo Eslabón,Canal 3,Fecha,Venta
7,100481,CL RETAIL,2018-01-01,63.0


In [5]:
data.shape

(853092, 4)

## dividir el df en 2 partes para entrenar modelos

In [6]:
# eslabones_unicos = data['Ultimo Eslabón'].unique()
# mitad_eslabones = eslabones_unicos[:len(eslabones_unicos)//2]
# data = data[data['Ultimo Eslabón'].isin(mitad_eslabones)]
# data.head()


# ARIMA SMOOTH_OPTIMISTA

In [6]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import os
from datetime import datetime

data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')

predictions = []

for id_value in data['ID'].unique():
    data_id = data[data['ID'] == id_value]
    data_id.set_index('Fecha', inplace=True, drop=True)
    data_id.index.freq = 'MS'
    y = data_id['Venta']
    
    if y.isnull().any() or np.isinf(y).any():
        print(f'Serie temporal con valores nulos o infinitos para ID: {id_value}')
        continue
    
    try:
        model = ARIMA(y, order=(5, 1, 0))
        model_fit = model.fit()
        future_predictions = model_fit.forecast(steps=12)
        future_dates = pd.date_range(start='2024-11-01', periods=12, freq='MS').strftime('%Y-%m-%d')
        
        for date, pred in zip(future_dates, future_predictions):
            predictions.append({
                'ID': id_value,
                'Fecha': date,
                'Prediccion_Venta': pred
            })
    except Exception as e:
        print(f'Error al ajustar el modelo para ID: {id_value} - {e}')
        continue

predictions_df = pd.DataFrame(predictions)

current_month = datetime.now().strftime('%Y-%m')
output_dir = os.path.expanduser(r'~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\3.Predecir\Methods\ML')
output_file = os.path.join(output_dir, f'ARIMA_SMOOTH_normal_{current_month}_e1.csv')
predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')

c:\Users\etorres.DERCOPARTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\etorres.DERCOPARTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\etorres.DERCOPARTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\etorres.DERCOPARTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization f

OSError: Cannot save file into a non-existent directory: 'C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\3.Predecir\Methods\ML'

In [7]:
output_dir = os.path.expanduser(r'~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda')
output_file = os.path.join(output_dir, f'ARIMA_SMOOTH_normal_{current_month}_e1.csv')
predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')

#AUTO ARIMA SMOOTH 

In [ ]:
# import pandas as pd
# import numpy as np
# from statsmodels.tsa.arima.model import ARIMA
# import os
# from pmdarima import auto_arima
# from datetime import datetime
# from numpy.linalg import LinAlgError
# import warnings

# # Supongamos que 'data' es tu DataFrame inicial
# # Asegúrate de que 'data' tenga las columnas 'Ultimo Eslabón', 'Canal 3', 'Fecha' y 'Venta'

# # Crear el ID único combinando 'Ultimo Eslabón' y 'Canal 3'
# data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
# data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')

# predictions = []

# # Iterar sobre cada ID único
# for id_value in data['ID'].unique():
#     data_id = data[data['ID'] == id_value].copy()
#     data_id.set_index('Fecha', inplace=True)
    
#     # Ordenar el índice y verificar la frecuencia
#     data_id = data_id.sort_index()
#     data_id = data_id.asfreq('MS')  # Asegura frecuencia mensual
    
#     y = data_id['Venta']
    
#     # Verificar si hay datos insuficientes
#     if y.isnull().all() or len(y.dropna()) < 3:
#         # Proyectar 0 si no hay datos suficientes
#         future_dates = pd.date_range(start='2024-10-01', periods=12, freq='MS')
#         for date in future_dates:
#             predictions.append({
#                 'ID': id_value,
#                 'Fecha': date.strftime('%Y-%m-%d'),
#                 'Prediccion_Venta': 0
#             })
#         continue
    
#     # Manejo de errores y advertencias
#     with warnings.catch_warnings():
#         warnings.filterwarnings("ignore")
#         try:
#             # Ajustar el modelo ARIMA
#             model = auto_arima(
#                 y,
#                 start_p=0, max_p=3,
#                 start_q=0, max_q=3,
#                 seasonal=False,
#                 stepwise=True,
#                 suppress_warnings=True,
#                 error_action='ignore',
#                 trace=False
#             )
#             # Predecir los próximos 12 meses
#             future_predictions = model.predict(n_periods=12)
#             future_dates = pd.date_range(start='2024-10-01', periods=12, freq='MS')
            
#             for date, pred in zip(future_dates, future_predictions):
#                 predictions.append({
#                     'ID': id_value,
#                     'Fecha': date.strftime('%Y-%m-%d'),
#                     'Prediccion_Venta': max(0, pred)  # Asegurar que la predicción no sea negativa
#                 })
#         except (LinAlgError, ValueError, Exception):
#             # Si ocurre un error, proyectar 0 para los próximos 12 meses
#             future_dates = pd.date_range(start='2024-10-01', periods=12, freq='MS')
#             for date in future_dates:
#                 predictions.append({
#                     'ID': id_value,
#                     'Fecha': date.strftime('%Y-%m-%d'),
#                     'Prediccion_Venta': 0
#                 })
#             continue

# # Crear el DataFrame de predicciones
# predictions_df = pd.DataFrame(predictions)

# # Guardar el archivo CSV
# current_month = datetime.now().strftime('%Y-%m')
# output_dir = os.path.expanduser(r'~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\3.Predecir\Methods\ML')
# output_file = os.path.join(output_dir, f'ARIMA_SMOOTH_AUTO_O1{current_month}.csv')

# # Asegurarse de que el directorio existe
# os.makedirs(output_dir, exist_ok=True)

# # Guardar las predicciones en un archivo CSV
# predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')


# ARIMA - ERRATIC - PRESENTE VIRTUAL

In [ ]:
data = melted_data[melted_data['Demand Type'] == 'Erratic']
data = data[['Ultimo Eslabón', 'Canal 3', 'Fecha', 'Venta']]
data.head(2)

AUTO_ARIMA

In [ ]:
# import pandas as pd
# import numpy as np
# from statsmodels.tsa.arima.model import ARIMA
# import os
# from pmdarima import auto_arima
# from datetime import datetime

# data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
# data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')

# predictions = []

# for id_value in data['ID'].unique():
#     data_id = data[data['ID'] == id_value]
#     data_id.set_index('Fecha', inplace=True, drop=True)
#     data_id.index.freq = 'MS'
#     y = data_id['Venta']
    
#     if y.isnull().any() or np.isinf(y).any():
#         continue
    
#     try:
#         model = auto_arima(y, seasonal=False, stepwise=True, suppress_warnings=True)
#         future_predictions = model.predict(n_periods=12)
#         future_dates = pd.date_range(start='2024-10-01', periods=12, freq='MS').strftime('%Y-%m-%d')
        
#         for date, pred in zip(future_dates, future_predictions):
#             predictions.append({
#                 'ID': id_value,
#                 'Fecha': date,
#                 'Prediccion_Venta': pred
#             })
#     except Exception:
#         continue

# predictions_df = pd.DataFrame(predictions)

# current_month = datetime.now().strftime('%Y-%m')
# output_dir = os.path.expanduser(r'~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\3.Predecir\Methods\ML')
# output_file = os.path.join(output_dir, f'AUTO_ARIMA_ERRATIC_{current_month}.csv')

# predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')


ARIMA

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import os
from datetime import datetime

data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')

predictions = []

for id_value in data['ID'].unique():
    data_id = data[data['ID'] == id_value]
    data_id.set_index('Fecha', inplace=True, drop=True)
    data_id.index.freq = 'MS'
    y = data_id['Venta']
    
    if y.isnull().any() or np.isinf(y).any():
        print(f'Serie temporal con valores nulos o infinitos para ID: {id_value}')
        continue
    
    try:
        model = ARIMA(y, order=(5, 1, 0))
        model_fit = model.fit()
        future_predictions = model_fit.forecast(steps=12)
        future_dates = pd.date_range(start='2024-11-01', periods=12, freq='MS').strftime('%Y-%m-%d')
        
        for date, pred in zip(future_dates, future_predictions):
            predictions.append({
                'ID': id_value,
                'Fecha': date,
                'Prediccion_Venta': pred
            })
    except Exception as e:
        print(f'Error al ajustar el modelo para ID: {id_value} - {e}')
        continue

predictions_df = pd.DataFrame(predictions)

current_month = datetime.now().strftime('%Y-%m')
output_dir = os.path.expanduser(r'~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\3.Predecir\Methods\ML')
output_file = os.path.join(output_dir, f'ARIMA_SMOOTH_Erratic_{current_month}_1.csv')

predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')


# ARIMA INTTERMITENT

In [ ]:
data = melted_data[melted_data['Demand Type'] == 'Intermittent']
data = data[['Ultimo Eslabón', 'Canal 3', 'Fecha', 'Venta']]
data

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import os
from datetime import datetime

data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')

predictions = []

for id_value in data['ID'].unique():
    data_id = data[data['ID'] == id_value]
    data_id.set_index('Fecha', inplace=True, drop=True)
    data_id.index.freq = 'MS'
    y = data_id['Venta']
    
    if y.isnull().any() or np.isinf(y).any():
        print(f'Serie temporal con valores nulos o infinitos para ID: {id_value}')
        continue
    
    try:
        model = ARIMA(y, order=(5, 1, 0))
        model_fit = model.fit()
        future_predictions = model_fit.forecast(steps=12)
        future_dates = pd.date_range(start='2024-11-01', periods=12, freq='MS').strftime('%Y-%m-%d')
        
        for date, pred in zip(future_dates, future_predictions):
            predictions.append({
                'ID': id_value,
                'Fecha': date,
                'Prediccion_Venta': pred
            })
    except Exception as e:
        print(f'Error al ajustar el modelo para ID: {id_value} - {e}')
        continue

predictions_df = pd.DataFrame(predictions)

current_month = datetime.now().strftime('%Y-%m')
output_dir = os.path.expanduser(r'~\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\3.Predecir\Methods\ML')
output_file = os.path.join(output_dir, f'ARIMA_SMOOTH_Intermittent_{current_month}_1.csv')

predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')


# CROSTON

In [ ]:
# data = melted_data[melted_data['Demand Type'] == 'Intermittent']
# data = data[['Ultimo Eslabón', 'Canal 3', 'Fecha', 'Venta']]
# data

In [ ]:
# import pandas as pd
# import numpy as np
# import os

# def croston(ts, alpha=0.4, h=12):
#     ts = np.array(ts)
#     n = len(ts)
#     a, p = np.zeros(n), np.zeros(n)
#     q = np.zeros(n)
#     y_hat = np.zeros(n + h)
#     a[0], p[0], q[0] = ts[0], 1, ts[0]
    
#     for t in range(1, n):
#         if ts[t] > 0:
#             p[t] = alpha * ts[t] + (1 - alpha) * p[t - 1]
#             q[t] = alpha * (t - a[t - 1]) + (1 - alpha) * q[t - 1]
#             a[t] = t
#         else:
#             p[t] = p[t - 1]
#             q[t] = q[t - 1]
#             a[t] = a[t - 1]
    
#     for t in range(n, n + h):
#         y_hat[t] = p[-1] / q[-1]
    
#     return y_hat[-h:]

# # Asegúrate de que 'data' es un DataFrame original y no una vista de otro DataFrame
# data = data.copy()

# # Crear la columna 'ID' usando .loc para evitar el SettingWithCopyWarning
# data.loc[:, 'ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)

# # Convertir la columna 'Fecha' al formato datetime usando .loc
# data.loc[:, 'Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')

# predictions = []

# # Simulamos estar en diciembre de 2023 pero queremos predecir a partir de enero de 2024
# for id_value in data['ID'].unique():
#     data_id = data[data['ID'] == id_value].copy()  # Asegurarse de trabajar con una copia del DataFrame
#     data_id.set_index('Fecha', inplace=True, drop=True)
#     data_id.index.freq = 'MS'  # Esto supone que las fechas son mensuales y consecutivas
#     y = data_id['Venta']
    
#     if y.isnull().any() or np.isinf(y).any():
#         print(f'Serie temporal con valores nulos o infinitos para ID: {id_value}')
#         continue
    
#     try:
#         # Aplicar el método de Croston
#         future_predictions = croston(y.values, alpha=0.4, h=12)
        
#         # Crear las fechas de predicción a partir de enero de 2024
#         future_dates = pd.date_range(start='2024-01-01', periods=12, freq='MS').strftime('%Y-%m-%d')
        
#         for date, pred in zip(future_dates, future_predictions):
#             predictions.append({
#                 'ID': id_value,
#                 'Fecha': date,
#                 'Prediccion_Venta': pred
#             })
#     except Exception as e:
#         print(f'Error al ajustar el modelo para ID: {id_value} - {e}')
#         continue

# predictions_df = pd.DataFrame(predictions)
# output_dir = r'C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo'
# output_file = os.path.join(output_dir, 'CROSTON_INTERMITTENT.csv')
# predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')

# print(f'Predicciones guardadas en {output_file}')
